# df_brd_w_prd_v2에서 유실값 확인

In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
from tqdm import trange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [13]:
df_brd_w_prd_v1_drop_lg = pd.read_csv("df_brd_w_prd_v1_drop_lg.csv")
df_brd_w_prd_v1_drop_lg = df_brd_w_prd_v1_drop_lg.iloc[:, 1:]

In [14]:
def get_search_len(brd_name):

    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)
        
        html = wd.page_source
        soup = BeautifulSoup(html, "html.parser")

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # ====

        #.products__product
        #.product
        # ====

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r
        
        try:
            while True:

                product_lst = parsing_html_with_class()
                
                global product_len
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    html = wd.page_source
                    soup = BeautifulSoup(html, "html.parser")
                    prd_len = len(soup.find_all('div','products__product product'))
                    return prd_len
                    break


        except NoSuchElementException:
            html = wd.page_source
            soup = BeautifulSoup(html, "html.parser")
            prd_len = len(soup.find_all('div','products__product product'))
            return prd_len
        
    except Exception as e:
        exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass

In [17]:
exception = []

In [16]:
df_brd_w_prd_v1_drop_lg

,brand_code_x,brand_name,product_link,product_name,brand_code_y,category,crawling_prd_cnt
0,https://www.glowpick.com/brands/8016,가가알로에,['https://www.glowpick.com/products/104713'],['라율 나이트 케어 너리싱 젤'],https://www.glowpick.com/brands/8016,['스킨케어'],1.0
1,https://www.glowpick.com/brands/2077,가네보,"['https://www.glowpick.com/products/101993', '...","['페이스업 파우더 밀라노 콜렉션 2018 [SPF14/PA+]', '코프레도르 스...",https://www.glowpick.com/brands/2077,"['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '...",12.0
2,https://www.glowpick.com/brands/8724,가드미,"['https://www.glowpick.com/products/112824', '...","['큐티클 클리너', '네일 큐티클 호호바 오일', '[단종] 슈퍼 단백질 헤어가...",https://www.glowpick.com/brands/8724,['네일'],3.0
3,https://www.glowpick.com/brands/10352,가든 오브 라이프,"['https://www.glowpick.com/products/133823', '...","['비타민 로우 B 컴플렉스', '로우 프로바이오틱스 장케어', '무가공 비타민C'...",https://www.glowpick.com/brands/10352,['이너뷰티'],12.0
4,https://www.glowpick.com/brands/7314,가디스가든,"['https://www.glowpick.com/products/96142', 'h...","['유기농 드림 리페어 선 리페어 나이트 크림', '유기농 데이 언던 선 리페어 세...",https://www.glowpick.com/brands/7314,"['스킨케어', '베이비', '립메이크업', '클렌징', '선케어']",14.0
...,...,...,...,...,...,...,...
7569,https://www.glowpick.com/brands/490,4711,"['https://www.glowpick.com/products/1622', 'ht...","['아쿠아 콜로니아 블러드 오렌지 & 바즐 오 드 코롱', '누보 콜로뉴 오드코롱'...",https://www.glowpick.com/brands/490,['프래그런스'],16.0
7570,https://www.glowpick.com/brands/10747,5.2뉴트리션,['https://www.glowpick.com/products/138092'],['다이어트 베이스 [초코맛]'],https://www.glowpick.com/brands/10747,['이너뷰티'],1.0
7571,https://www.glowpick.com/brands/10183,76N1,"['https://www.glowpick.com/products/140014', '...","['비 해피 더블 액션 미스트', '비 해피 오일 [대마씨 & 동백]', '슈퍼듀퍼...",https://www.glowpick.com/brands/10183,"['스킨케어', '클렌징', '마스크/팩']",8.0
7572,https://www.glowpick.com/brands/6814,87발렌타인,['https://www.glowpick.com/products/89547'],['이상훈의 제모크림'],https://www.glowpick.com/brands/6814,['배쓰&바디'],1.0


In [ ]:
for i in tqdm(list(df_brd_w_prd_v1_drop_lg[6000:].index)):
    brd_name = df_brd_w_prd_v1_drop_lg.loc[i, 'brand_name']
    prd_len = get_search_len(brd_name)
    df_brd_w_prd_v1_drop_lg.loc[i, 'prd_len'] = prd_len

  0%|          | 0/1574 [00:00<?, ?it/s]



====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1468.0 Safari/537.36


Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36


...



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


Mozilla/5.0 (Windows NT 5.1; rv:21.0) Gecko/20130331 Firefox/21.0




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [37]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999 = pd.read_csv("find_missing_df_brd_w_prd_v1_drop_lg_0_999.csv")
find_missing_df_brd_w_prd_v1_drop_lg_0_999 = find_missing_df_brd_w_prd_v1_drop_lg_0_999.iloc[:, 1:]

In [38]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999.columns

Index(['brand_code_x', 'brand_name', 'product_link', 'product_name',
       'brand_code_y', 'category', 'crawling_prd_cnt', 'prd_len'],
      dtype='object')

In [40]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999 = find_missing_df_brd_w_prd_v1_drop_lg_0_999.drop(["brand_code_y"], axis="columns")

In [42]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999.rename(index = {'brand_code_x': 'brand_code'}, inplace = True)

In [43]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999

,brand_code_x,brand_name,product_link,product_name,category,crawling_prd_cnt,prd_len
0,https://www.glowpick.com/brands/8016,가가알로에,['https://www.glowpick.com/products/104713'],['라율 나이트 케어 너리싱 젤'],['스킨케어'],1.0,1.0
1,https://www.glowpick.com/brands/2077,가네보,"['https://www.glowpick.com/products/101993', '...","['페이스업 파우더 밀라노 콜렉션 2018 [SPF14/PA+]', '코프레도르 스...","['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '...",12.0,12.0
2,https://www.glowpick.com/brands/8724,가드미,"['https://www.glowpick.com/products/112824', '...","['큐티클 클리너', '네일 큐티클 호호바 오일', '[단종] 슈퍼 단백질 헤어가...",['네일'],3.0,3.0
3,https://www.glowpick.com/brands/10352,가든 오브 라이프,"['https://www.glowpick.com/products/133823', '...","['비타민 로우 B 컴플렉스', '로우 프로바이오틱스 장케어', '무가공 비타민C'...",['이너뷰티'],12.0,12.0
4,https://www.glowpick.com/brands/7314,가디스가든,"['https://www.glowpick.com/products/96142', 'h...","['유기농 드림 리페어 선 리페어 나이트 크림', '유기농 데이 언던 선 리페어 세...","['스킨케어', '베이비', '립메이크업', '클렌징', '선케어']",14.0,14.0
...,...,...,...,...,...,...,...
995,https://www.glowpick.com/brands/710,듀이트리,"['https://www.glowpick.com/products/59659', 'h...","['AC 컨트롤 딥마스크', '하이 아미노 올 클렌저', '픽 앤 퀵 카밍 풀 마스...","['스킨케어', '페이스메이크업', '베이비', '남성화장품', '아이메이크업', ...",242.0,242.0
996,https://www.glowpick.com/brands/448,듀크레이,"['https://www.glowpick.com/products/52821', 'h...","['케라크닐 리페어 크림', '덱시안 크렘 에몰리앙뜨 안티 그라타지 (덱시안 크림)...","['스킨케어', '페이스메이크업', '립메이크업', '클렌징', '마스크/팩', '...",38.0,38.0
997,https://www.glowpick.com/brands/10069,드기,"['https://www.glowpick.com/products/129763', '...","['퓨어샴푸', '에너지샴푸', '클린업샴푸']",['헤어'],3.0,3.0
998,https://www.glowpick.com/brands/588,드끌레오,"['https://www.glowpick.com/products/16050', 'h...","['', '', '', '', '', '', '', '', '', '', '', '...","['스킨케어', '페이스메이크업', '립메이크업', '클렌징', '마스크/팩', '...",76.0,76.0


In [49]:
exception = list(find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[find_missing_df_brd_w_prd_v1_drop_lg_0_999["prd_len"].isna(), 'brand_name'])

In [56]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[find_missing_df_brd_w_prd_v1_drop_lg_0_999['brand_name'] == "드라니코", :].index, 'prd_len']

999    1.0
Name: prd_len, dtype: float64

In [58]:
for i in tqdm(exception):
    brd_name = i
    prd_len = get_search_len(brd_name)
    find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[find_missing_df_brd_w_prd_v1_drop_lg_0_999['brand_name'] == brd_name, :].index, 'prd_len'] = prd_len

  0%|          | 0/16 [00:00<?, ?it/s]



====== WebDriver manager ======


Mozilla/5.0 (X11; NetBSD) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17


...

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:29.0) Gecko/20120101 Firefox/29.0


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.62 Safari/537.36


Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17


In [69]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999

,brand_code_x,brand_name,product_link,product_name,category,crawling_prd_cnt,prd_len
0,https://www.glowpick.com/brands/8016,가가알로에,['https://www.glowpick.com/products/104713'],['라율 나이트 케어 너리싱 젤'],['스킨케어'],1.0,1.0
1,https://www.glowpick.com/brands/2077,가네보,"['https://www.glowpick.com/products/101993', '...","['페이스업 파우더 밀라노 콜렉션 2018 [SPF14/PA+]', '코프레도르 스...","['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '...",12.0,12.0
2,https://www.glowpick.com/brands/8724,가드미,"['https://www.glowpick.com/products/112824', '...","['큐티클 클리너', '네일 큐티클 호호바 오일', '[단종] 슈퍼 단백질 헤어가...",['네일'],3.0,3.0
3,https://www.glowpick.com/brands/10352,가든 오브 라이프,"['https://www.glowpick.com/products/133823', '...","['비타민 로우 B 컴플렉스', '로우 프로바이오틱스 장케어', '무가공 비타민C'...",['이너뷰티'],12.0,12.0
4,https://www.glowpick.com/brands/7314,가디스가든,"['https://www.glowpick.com/products/96142', 'h...","['유기농 드림 리페어 선 리페어 나이트 크림', '유기농 데이 언던 선 리페어 세...","['스킨케어', '베이비', '립메이크업', '클렌징', '선케어']",14.0,14.0
...,...,...,...,...,...,...,...
995,https://www.glowpick.com/brands/710,듀이트리,"['https://www.glowpick.com/products/59659', 'h...","['AC 컨트롤 딥마스크', '하이 아미노 올 클렌저', '픽 앤 퀵 카밍 풀 마스...","['스킨케어', '페이스메이크업', '베이비', '남성화장품', '아이메이크업', ...",242.0,242.0
996,https://www.glowpick.com/brands/448,듀크레이,"['https://www.glowpick.com/products/52821', 'h...","['케라크닐 리페어 크림', '덱시안 크렘 에몰리앙뜨 안티 그라타지 (덱시안 크림)...","['스킨케어', '페이스메이크업', '립메이크업', '클렌징', '마스크/팩', '...",38.0,38.0
997,https://www.glowpick.com/brands/10069,드기,"['https://www.glowpick.com/products/129763', '...","['퓨어샴푸', '에너지샴푸', '클린업샴푸']",['헤어'],3.0,3.0
998,https://www.glowpick.com/brands/588,드끌레오,"['https://www.glowpick.com/products/16050', 'h...","['', '', '', '', '', '', '', '', '', '', '', '...","['스킨케어', '페이스메이크업', '립메이크업', '클렌징', '마스크/팩', '...",76.0,76.0


In [76]:
cnt = 0

for i in range(len(find_missing_df_brd_w_prd_v1_drop_lg_0_999)):
    if find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[i, 'crawling_prd_cnt'] < find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[i, 'prd_len']:
        print(find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[i, :])
        cnt += 1

brand_code_x                     https://www.glowpick.com/brands/7406
brand_name                                                       고려은단
product_link        ['https://www.glowpick.com/products/97645', 'h...
product_name        ['비타민C 1000', '비타민C 골드 플러스', '비타민C 1000 이지', '...
category                                            ['이너뷰티', '마스크/팩']
crawling_prd_cnt                                                 17.0
prd_len                                                          18.0
Name: 32, dtype: object
brand_code_x                      https://www.glowpick.com/brands/933
brand_name                                                         그램
product_link        ['https://www.glowpick.com/products/24009', 'h...
product_name        ['더 촉촉한 크림 50그램', '더 화사한 크림 50', '더블케미 반반팩 2.0...
category                                            ['스킨케어', '마스크/팩']
crawling_prd_cnt                                                233.0
prd_len                                                         23

In [75]:
cnt

3

In [ ]:
find_missing_df_brd_w_prd_v1_drop_lg_0_999.loc[771, 'crawling_prd_cnt']

---

In [102]:
v4 = pd.read_csv("v4_update.csv")
v4 = v4.iloc[:, 1:]

In [103]:
v4_prd_link_lst = list(v4["제품 link"])

In [104]:
len(list(set(v4_prd_link_lst)))

64167

In [107]:
set(list(v4.loc[v4.duplicated(['제품 link']), "브랜드명"]))

{'JM솔루션',
 '끌라삐엘',
 '닐스야드 레머디스',
 '닥터레미듀',
 '닥터헤디슨',
 '더블유드레스룸',
 '더샘',
 '더유핏',
 '데저트에센스',
 '드끌레오',
 '라노아',
 '라뮤즈',
 '로지컬리스킨',
 '린제이',
 '마몽드',
 '무슈제이',
 '미쉘메르시에',
 '버츠비',
 '벤튼',
 '벨',
 '보나쥬르',
 '보니앤제이',
 '비바이바닐라',
 '산다화',
 '새터데이 스킨',
 '설화수',
 '세바메드',
 '세인트이브스',
 '세타필',
 '소티스',
 '솔더랩',
 '시크릿키',
 '싸이닉',
 '아로마티카',
 '아크네스',
 '야다',
 '어뮤즈',
 '에뛰드',
 '에버에센스',
 '올리브영',
 '이니스프리',
 '이슬품은나',
 '이아소',
 '자연담은유리병',
 '자연의벗',
 '자이모겐',
 '카이',
 '코츠비',
 '킵쿨',
 '포렌코즈',
 '폴텐',
 '프리메라',
 '플루',
 '피토레스코',
 '필링빈',
 '한스킨',
 '헤라',
 '휘게'}

In [108]:
v4_old = pd.read_csv("V4_DF_ind_set.csv")
v4_old = v4_old.iloc[:, 1:]

In [109]:
v4_old_brd_lst = list(set(list(v4_old["브랜드명"])))

In [110]:
len(v4_old_brd_lst)

4134

In [113]:
#v4에서 상품이 중복된 경우의 수 파악
dup_brd_lst = []
for brd in tqdm(v4_old_brd_lst):
    if len(list(v4_old.loc[v4_old["브랜드명"]== brd, "제품명"])) != len(list(set(list(v4_old.loc[v4_old["브랜드명"]== brd, "제품명"])))):
        dup_brd_lst.append(v4_old.loc[v4_old["브랜드명"]== brd, "브랜드명"])

  0%|          | 0/4134 [00:00<?, ?it/s]

In [128]:
list(dup_brd_lst[1])[0]

'비프루브'

In [129]:
dup_brd_lst_ = []
for i in range(len(dup_brd_lst)):
    dup_brd_lst_.append(list(dup_brd_lst[i])[0])

In [131]:
dup_brd_lst = dup_brd_lst_.copy()

In [132]:
dup_brd_lst

['에센스',
 '비프루브',
 '티엔',
 '아임미미',
 '밀크바오밥',
 '딥티크',
 '이니스프리',
 '네오젠 더마로지',
 '엘프 코스메틱스',
 '이즈앤트리',
 '킵쿨',
 '참존',
 '바이빠세',
 '쏘내추럴',
 '리얼베리어',
 '미쟝센',
 '네이처리퍼블릭',
 '한스킨',
 '닥터제이준',
 '아리따움',
 '라뮤즈',
 '빌리프',
 '에버에센스',
 '나인위시스',
 '조성아뷰티',
 '낫포유',
 '파티온',
 '세포라 컬렉션',
 '누니',
 'Dr.G',
 '스킨톡',
 '벨라',
 '비오템옴므',
 '잇츠스킨',
 '메이크프렘',
 '랑콤',
 '웨이크메이크',
 '홀리카홀리카',
 '이지피지',
 '토니모리',
 '카일라스',
 '센카',
 '유리피부',
 '올리브영',
 '에스쁘아',
 '뉴트로지나',
 '포인트',
 '록시땅',
 '파파레서피',
 '조성아22',
 '비욘드',
 '잉가',
 '닥터뉴엘',
 '에이페',
 '설화수',
 '마몽드',
 '미미로린스',
 '미쉘메르시에',
 '자트인사이트',
 '씨앤트리',
 '롭스',
 'AHC',
 '미샤',
 '바닐라코',
 '올리베리어',
 '비오레',
 '키핀터치',
 '어퓨',
 '제이준',
 '미구하라',
 '더스킨하우스',
 '더블유드레스룸',
 '오휘',
 '시세이도',
 '마녀공장',
 '데메테르',
 '닥터알카이티스',
 '리더스코스메틱',
 '비디비치',
 '원데이즈유',
 '바비브라운',
 '샬랑드파리',
 '코츠비',
 '온더바디',
 '진',
 '셀퓨전씨',
 '애플캔들',
 '구달',
 '아틀리에코롱',
 '보니앤제이',
 '클리오',
 '케라스타즈',
 '벨',
 '피카소',
 '쎄라덤',
 '셀본',
 '아크웰',
 '해피바스',
 '더바디샵',
 '에뛰드',
 '오바코',
 '화이트',
 '네오젠 코드나인',
 '헤라',
 '엔프라니',
 '아윤채',
 '쌍빠',
 '러쉬',
 '스킨푸드',
 '싸이닉',
 '투쿨포스쿨',
 '지나인스킨',

In [133]:
v4_3 = pd.read_csv("glow_pick_product_info_v4.3.csv")
v4_3 = v4_3.iloc[:, 1:]

In [134]:
v4_3_brd_lst = list(set(list(v4_3["브랜드명"])))

In [139]:
len(v4_3_brd_lst)

1471

In [140]:
#v4에서 상품이 중복된 경우의 수 파악
dup_brd_lst_v4_3 = []
for brd in tqdm(v4_3_brd_lst):
    if len(list(v4_3.loc[v4_3["브랜드명"]== brd, "제품명"])) != len(list(set(list(v4_3.loc[v4_3["브랜드명"]== brd, "제품명"])))):
        dup_brd_lst_v4_3.append(v4_3.loc[v4_old["브랜드명"]== brd, "브랜드명"])

  0%|          | 0/1471 [00:00<?, ?it/s]

In [155]:
dup_brd_lst_v4_3_ = []
for i in range(len(dup_brd_lst_v4_3)):
    try:
        dup_brd_lst_v4_3_.append(list(dup_brd_lst_v4_3[i])[0])
    except:
        pass

In [156]:
len(dup_brd_lst_v4_3_)

117

In [159]:
list(set(dup_brd_lst_v4_3_))

['스와니코코',
 '믹순',
 '비디비치',
 '메디힐',
 '미쟝센',
 '맥',
 '에이프릴스킨',
 '비욘드',
 '네이처리퍼블릭',
 '아이다코스메틱',
 'VT COSMETICS',
 '더블유드레스룸',
 '해피바스',
 '아이오페',
 '구찌',
 'JM솔루션',
 '정샘물',
 '편강율',
 '이글립스',
 '아임미미',
 '쿠모',
 '라우쉬',
 '그레이멜린',
 '에뛰드',
 '손앤박',
 '한스킨',
 '더마필리아',
 '올리브영',
 '맥퀸뉴욕',
 '지투셀',
 '프레시팝',
 '에스쁘아',
 '미샤',
 '디오프러스',
 '바닐라코',
 '수려한',
 '아로마티카',
 '땡큐파머',
 '부르조아',
 '헤라',
 '일리윤',
 '더프트앤도프트',
 'W.피부연구소',
 '이자녹스',
 '벨라몬스터',
 '3CE',
 '딥티크',
 '로하셀',
 '아크네스',
 '시세이도',
 '살바토레 페라가모',
 '이니스프리',
 '숨37˚',
 '라뮤즈',
 '클리오',
 '아모레퍼시픽',
 '잇츠스킨',
 '캐롤프랑크',
 '랑콤',
 '빌리프',
 '후르디아',
 '더샘',
 '투쿨포스쿨',
 '23 years old',
 '라비오뜨',
 '베네피트',
 '디어달리아',
 '케어존',
 '엘라스틴',
 '블리블리',
 '어퓨',
 '스킨푸드',
 '안나수이',
 '뷰티모리',
 '홀리카홀리카',
 '오르템',
 '페리페라',
 '톤28',
 '아크메르',
 '온더바디',
 '실크테라피',
 '세이어스',
 '아워글래스',
 '미미로린스',
 '데저트에센스',
 '센텔리안24',
 '토니모리',
 '바디판타지',
 '문샷',
 '쏘내추럴',
 '프레쉬',
 'CNP',
 '디폰데']